# Heart Disease Prediction Analysis

## CS445 - Artificial Intelligence Project

This notebook follows the CRISP-DM methodology to build a heart disease prediction model using the UCI Cleveland Heart Disease dataset.

**Author:** Jaron Cabral, Hunter 
**Dataset:** UCI Cleveland Heart Disease Dataset (303 records, 14 attributes)

---
## 1. Business Understanding

Early detection of heart disease is extremely important because cardiovascular conditions often progress silently until they cause severe, sometimes fatal, events such as heart attacks or heart failure.

**Project Goals:**
- Identify which clinical and lifestyle indicators most strongly predict heart disease
- Build and compare tree-based models (Decision Tree and Random Forest) to classify patients into "disease" vs "no disease" categories
- Design a user-friendly Heart Risk Advisor that turns model outputs into understandable, educational feedback for non-experts

**Target Users:**
- Primary care providers seeking data-driven cardiovascular risk assessment tools
- Individuals who want a snapshot of their cardiovascular risk profile

---
## 2. Data Understanding

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib

import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Define column names for the Cleveland dataset
column_names = [
    'age',       # Age in years
    'sex',       # Sex (1 = male, 0 = female)
    'cp',        # Chest pain type (1-4)
    'trestbps',  # Resting blood pressure (mm Hg)
    'chol',      # Serum cholesterol (mg/dl)
    'fbs',       # Fasting blood sugar > 120 mg/dl (1 = true, 0 = false)
    'restecg',   # Resting ECG results (0-2)
    'thalach',   # Maximum heart rate achieved
    'exang',     # Exercise induced angina (1 = yes, 0 = no)
    'oldpeak',   # ST depression induced by exercise
    'slope',     # Slope of peak exercise ST segment (1-3)
    'ca',        # Number of major vessels colored by fluoroscopy (0-3)
    'thal',      # Thalassemia (3 = normal, 6 = fixed defect, 7 = reversible defect)
    'target'     # Heart disease (0 = no disease, 1-4 = disease severity)
]

# Load the dataset
df = pd.read_csv('processed.cleveland.data', names=column_names, na_values='?')

print(f"Dataset Shape: {df.shape}")
print(f"Number of records: {len(df)}")
print(f"Number of features: {len(df.columns) - 1}")

Dataset Shape: (303, 14)
Number of records: 303
Number of features: 13


In [3]:
# Display first few rows
df.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
5,56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,0
6,62.0,0.0,4.0,140.0,268.0,0.0,2.0,160.0,0.0,3.6,3.0,2.0,3.0,3
7,57.0,0.0,4.0,120.0,354.0,0.0,0.0,163.0,1.0,0.6,1.0,0.0,3.0,0
8,63.0,1.0,4.0,130.0,254.0,0.0,2.0,147.0,0.0,1.4,2.0,1.0,7.0,2
9,53.0,1.0,4.0,140.0,203.0,1.0,2.0,155.0,1.0,3.1,3.0,0.0,7.0,1


In [4]:
# Check data types and missing values
print("Data Types:")
print(df.dtypes)
print("\nMissing Values:")
print(df.isnull().sum())

Data Types:
age         float64
sex         float64
cp          float64
trestbps    float64
chol        float64
fbs         float64
restecg     float64
thalach     float64
exang       float64
oldpeak     float64
slope       float64
ca          float64
thal        float64
target        int64
dtype: object

Missing Values:
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          4
thal        2
target      0
dtype: int64


In [5]:
# Summary statistics
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,299.000000,301.000000,303.000000
mean,54.438944,0.679868,3.158416,131.689769,246.693069,0.148515,0.990099,149.607261,0.326733,1.039604,1.600660,0.672241,4.734219,0.937294
std,9.038662,0.467299,0.960126,17.599748,51.776918,0.356198,0.994971,22.875003,0.469794,1.161075,0.616226,0.937438,1.939706,1.228536
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
25%,48.000000,0.000000,3.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
50%,56.000000,1.000000,3.000000,130.000000,241.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000,3.000000,0.000000
75%,61.000000,1.000000,4.000000,140.000000,275.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000,7.000000,2.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,7.000000,4.000000


In [6]:
# Target distribution (before binarization)
print("Target Variable Distribution (Original):")
print(df['target'].value_counts().sort_index())

Target Variable Distribution (Original):
target
0    164
1     55
2     36
3     35
4     13
Name: count, dtype: int64


### Interactive Visualizations (Plotly)

In [7]:
# VISUALIZATION 1: Correlation Heatmap
# Create a copy for visualization (handle missing values first)
df_viz = df.copy()
df_viz['ca'] = df_viz['ca'].fillna(df_viz['ca'].mode()[0])
df_viz['thal'] = df_viz['thal'].fillna(df_viz['thal'].mode()[0])
df_viz['disease'] = (df_viz['target'] > 0).astype(int)  # Binarize for visualization

# Calculate correlation matrix
corr_matrix = df_viz.drop('target', axis=1).corr()

fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='RdBu_r',
    zmid=0,
    text=np.round(corr_matrix.values, 2),
    texttemplate='%{text}',
    textfont={"size": 10},
    hoverongaps=False
))

fig.update_layout(
    title='Feature Correlation Heatmap',
    width=800,
    height=700,
    xaxis_title='Features',
    yaxis_title='Features'
)

fig.show()

In [8]:
# VISUALIZATION 2: Box Plots - Key Features by Disease Status
df_viz['Disease Status'] = df_viz['disease'].map({0: 'No Disease', 1: 'Disease'})

fig = make_subplots(rows=2, cols=2, subplot_titles=(
    'Cholesterol by Disease Status',
    'Resting Blood Pressure by Disease Status',
    'Max Heart Rate by Disease Status',
    'ST Depression (oldpeak) by Disease Status'
))

# Cholesterol
for status, color in [('No Disease', '#2ecc71'), ('Disease', '#e74c3c')]:
    data = df_viz[df_viz['Disease Status'] == status]['chol']
    fig.add_trace(go.Box(y=data, name=status, marker_color=color, showlegend=True), row=1, col=1)

# Resting Blood Pressure
for status, color in [('No Disease', '#2ecc71'), ('Disease', '#e74c3c')]:
    data = df_viz[df_viz['Disease Status'] == status]['trestbps']
    fig.add_trace(go.Box(y=data, name=status, marker_color=color, showlegend=False), row=1, col=2)

# Max Heart Rate
for status, color in [('No Disease', '#2ecc71'), ('Disease', '#e74c3c')]:
    data = df_viz[df_viz['Disease Status'] == status]['thalach']
    fig.add_trace(go.Box(y=data, name=status, marker_color=color, showlegend=False), row=2, col=1)

# ST Depression
for status, color in [('No Disease', '#2ecc71'), ('Disease', '#e74c3c')]:
    data = df_viz[df_viz['Disease Status'] == status]['oldpeak']
    fig.add_trace(go.Box(y=data, name=status, marker_color=color, showlegend=False), row=2, col=2)

fig.update_layout(
    title='Distribution of Key Features by Disease Status',
    height=700,
    width=900
)

fig.show()

In [9]:
# VISUALIZATION 3: Distribution of Key Features (Histograms)
fig = make_subplots(rows=2, cols=2, subplot_titles=(
    'Age Distribution',
    'Cholesterol Distribution',
    'Max Heart Rate Distribution',
    'ST Depression Distribution'
))

# Age
fig.add_trace(go.Histogram(x=df_viz['age'], nbinsx=20, name='Age', marker_color='#3498db'), row=1, col=1)

# Cholesterol
fig.add_trace(go.Histogram(x=df_viz['chol'], nbinsx=20, name='Cholesterol', marker_color='#9b59b6'), row=1, col=2)

# Max Heart Rate
fig.add_trace(go.Histogram(x=df_viz['thalach'], nbinsx=20, name='Max HR', marker_color='#e67e22'), row=2, col=1)

# ST Depression
fig.add_trace(go.Histogram(x=df_viz['oldpeak'], nbinsx=20, name='ST Depression', marker_color='#1abc9c'), row=2, col=2)

fig.update_layout(
    title='Distribution of Key Numeric Features',
    height=600,
    width=900,
    showlegend=False
)

fig.show()

In [10]:
# VISUALIZATION 4: Categorical Features Analysis
# Sex distribution by disease status
sex_disease = df_viz.groupby(['sex', 'Disease Status']).size().reset_index(name='count')
sex_disease['Sex'] = sex_disease['sex'].map({0: 'Female', 1: 'Male'})

fig1 = px.bar(sex_disease, x='Sex', y='count', color='Disease Status',
              barmode='group', title='Disease Distribution by Sex',
              color_discrete_map={'No Disease': '#2ecc71', 'Disease': '#e74c3c'})
fig1.update_layout(width=600, height=400)
fig1.show()

In [11]:
# Chest Pain Type distribution
cp_labels = {1: 'Typical Angina', 2: 'Atypical Angina', 3: 'Non-anginal Pain', 4: 'Asymptomatic'}
cp_disease = df_viz.groupby(['cp', 'Disease Status']).size().reset_index(name='count')
cp_disease['Chest Pain Type'] = cp_disease['cp'].map(cp_labels)

fig2 = px.bar(cp_disease, x='Chest Pain Type', y='count', color='Disease Status',
              barmode='group', title='Disease Distribution by Chest Pain Type',
              color_discrete_map={'No Disease': '#2ecc71', 'Disease': '#e74c3c'})
fig2.update_layout(width=700, height=400)
fig2.show()

---
## 3. Data Preparation

In [12]:
# Create a clean copy for modeling
df_model = df.copy()

print("Missing values before handling:")
print(df_model.isnull().sum()[df_model.isnull().sum() > 0])

Missing values before handling:
ca      4
thal    2
dtype: int64


In [13]:
# Handle missing values
# Using mode imputation for 'ca' and 'thal' as they are categorical features

print("Mode of 'ca':", df_model['ca'].mode()[0])
print("Mode of 'thal':", df_model['thal'].mode()[0])

df_model['ca'] = df_model['ca'].fillna(df_model['ca'].mode()[0])
df_model['thal'] = df_model['thal'].fillna(df_model['thal'].mode()[0])

print("\nMissing values after handling:")
print(df_model.isnull().sum().sum())

Mode of 'ca': 0.0
Mode of 'thal': 3.0

Missing values after handling:
0


In [14]:
# Binarize target variable
# Original: 0 = no disease, 1-4 = increasing severity
# Binarized: 0 = no disease, 1 = disease (any severity)

print("Original target distribution:")
print(df_model['target'].value_counts().sort_index())

df_model['target'] = (df_model['target'] > 0).astype(int)

print("\nBinarized target distribution:")
print(df_model['target'].value_counts())
print(f"\nDisease rate: {df_model['target'].mean()*100:.1f}%")

Original target distribution:
target
0    164
1     55
2     36
3     35
4     13
Name: count, dtype: int64

Binarized target distribution:
target
0    164
1    139
Name: count, dtype: int64

Disease rate: 45.9%


In [15]:
# Separate features and target
X = df_model.drop('target', axis=1)
y = df_model['target']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature names: {list(X.columns)}")

Features shape: (303, 13)
Target shape: (303,)

Feature names: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']


In [16]:
# Train/Test Split (80/20, stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {len(X_train)} ({len(X_train)/len(X)*100:.0f}%)")
print(f"Test set size: {len(X_test)} ({len(X_test)/len(X)*100:.0f}%)")
print(f"\nTraining disease rate: {y_train.mean()*100:.1f}%")
print(f"Test disease rate: {y_test.mean()*100:.1f}%")

Training set size: 242 (80%)
Test set size: 61 (20%)

Training disease rate: 45.9%
Test disease rate: 45.9%


### Data Preparation Summary

**Missing Values:**
- `ca` (major vessels): 4 missing values → filled with mode (0.0)
- `thal` (thalassemia): 2 missing values → filled with mode (3.0)
- **Rationale:** Mode imputation preserves the distribution for categorical features

**Categorical Encoding:**
- Features are already numerically encoded in the dataset
- `sex`: 0/1 (female/male)
- `cp`, `restecg`, `slope`, `thal`: ordinal integers preserved

**Target Binarization:**
- Original: 0 (no disease), 1-4 (disease severity levels)
- Binarized: 0 (no disease), 1 (any disease)

**Scaling:**
- NOT applied (tree-based models don't require scaled features)

---
## 4. Modeling

### Model 1: Decision Tree Classifier

In [17]:
# Default Decision Tree
dt_default = DecisionTreeClassifier(random_state=42)
dt_default.fit(X_train, y_train)

# Predictions
y_pred_dt_default = dt_default.predict(X_test)

# Metrics
dt_default_metrics = {
    'accuracy': accuracy_score(y_test, y_pred_dt_default),
    'precision': precision_score(y_test, y_pred_dt_default),
    'recall': recall_score(y_test, y_pred_dt_default),
    'f1': f1_score(y_test, y_pred_dt_default)
}

print("Decision Tree (Default) Performance:")
for metric, value in dt_default_metrics.items():
    print(f"  {metric.capitalize()}: {value:.4f}")

Decision Tree (Default) Performance:
  Accuracy: 0.7377
  Precision: 0.6765
  Recall: 0.8214
  F1: 0.7419


In [18]:
# Tuned Decision Tree (tuning max_depth)
param_grid_dt = {'max_depth': [3, 5, 7, 10, None]}

grid_search_dt = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid_dt,
    cv=5,
    scoring='f1',
    return_train_score=True
)
grid_search_dt.fit(X_train, y_train)

print("Decision Tree Hyperparameter Tuning Results:")
print(f"Best parameters: {grid_search_dt.best_params_}")
print(f"Best CV F1 score: {grid_search_dt.best_score_:.4f}")

Decision Tree Hyperparameter Tuning Results:
Best parameters: {'max_depth': 10}
Best CV F1 score: 0.7078


In [19]:
# Evaluate tuned Decision Tree
dt_tuned = grid_search_dt.best_estimator_
y_pred_dt_tuned = dt_tuned.predict(X_test)

dt_tuned_metrics = {
    'accuracy': accuracy_score(y_test, y_pred_dt_tuned),
    'precision': precision_score(y_test, y_pred_dt_tuned),
    'recall': recall_score(y_test, y_pred_dt_tuned),
    'f1': f1_score(y_test, y_pred_dt_tuned)
}

print("Decision Tree (Tuned) Performance:")
for metric, value in dt_tuned_metrics.items():
    print(f"  {metric.capitalize()}: {value:.4f}")

Decision Tree (Tuned) Performance:
  Accuracy: 0.7869
  Precision: 0.7143
  Recall: 0.8929
  F1: 0.7937


In [20]:
# Decision Tree Feature Importance
dt_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': dt_tuned.feature_importances_
}).sort_values('importance', ascending=True)

fig = px.bar(dt_importance, x='importance', y='feature', orientation='h',
             title='Decision Tree (Tuned) - Feature Importance',
             labels={'importance': 'Importance', 'feature': 'Feature'},
             color='importance', color_continuous_scale='Viridis')
fig.update_layout(height=500, width=700, showlegend=False)
fig.show()

### Model 2: Random Forest Classifier

In [21]:
# Tuned Random Forest (tuning n_estimators and max_depth)
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None]
}

grid_search_rf = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid_rf,
    cv=5,
    scoring='f1',
    return_train_score=True
)
grid_search_rf.fit(X_train, y_train)

print("Random Forest Hyperparameter Tuning Results:")
print(f"Best parameters: {grid_search_rf.best_params_}")
print(f"Best CV F1 score: {grid_search_rf.best_score_:.4f}")

Random Forest Hyperparameter Tuning Results:
Best parameters: {'max_depth': 5, 'n_estimators': 50}
Best CV F1 score: 0.8021


In [22]:
# Evaluate tuned Random Forest
rf_tuned = grid_search_rf.best_estimator_
y_pred_rf = rf_tuned.predict(X_test)

rf_metrics = {
    'accuracy': accuracy_score(y_test, y_pred_rf),
    'precision': precision_score(y_test, y_pred_rf),
    'recall': recall_score(y_test, y_pred_rf),
    'f1': f1_score(y_test, y_pred_rf)
}

print("Random Forest (Tuned) Performance:")
for metric, value in rf_metrics.items():
    print(f"  {metric.capitalize()}: {value:.4f}")

Random Forest (Tuned) Performance:
  Accuracy: 0.9016
  Precision: 0.8438
  Recall: 0.9643
  F1: 0.9000


In [23]:
# Random Forest Feature Importance
rf_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_tuned.feature_importances_
}).sort_values('importance', ascending=True)

fig = px.bar(rf_importance, x='importance', y='feature', orientation='h',
             title='Random Forest (Tuned) - Feature Importance',
             labels={'importance': 'Importance', 'feature': 'Feature'},
             color='importance', color_continuous_scale='Plasma')
fig.update_layout(height=500, width=700, showlegend=False)
fig.show()

### Model Comparison Table

In [24]:
# Create comparison table
comparison_df = pd.DataFrame({
    'Model': ['Decision Tree (Default)', 'Decision Tree (Tuned)', 'Random Forest (Tuned)'],
    'Accuracy': [dt_default_metrics['accuracy'], dt_tuned_metrics['accuracy'], rf_metrics['accuracy']],
    'Precision': [dt_default_metrics['precision'], dt_tuned_metrics['precision'], rf_metrics['precision']],
    'Recall': [dt_default_metrics['recall'], dt_tuned_metrics['recall'], rf_metrics['recall']],
    'F1 Score': [dt_default_metrics['f1'], dt_tuned_metrics['f1'], rf_metrics['f1']],
    'Notes': [
        'Baseline model',
        f'max_depth={grid_search_dt.best_params_["max_depth"]}',
        f'n_estimators={grid_search_rf.best_params_["n_estimators"]}, max_depth={grid_search_rf.best_params_["max_depth"]}'
    ]
})

# Format numeric columns
for col in ['Accuracy', 'Precision', 'Recall', 'F1 Score']:
    comparison_df[col] = comparison_df[col].apply(lambda x: f'{x:.4f}')

comparison_df

,Model,Accuracy,Precision,Recall,F1 Score,Notes
0,Decision Tree (Default),0.7377,0.6765,0.8214,0.7419,Baseline model
1,Decision Tree (Tuned),0.7869,0.7143,0.8929,0.7937,max_depth=10
2,Random Forest (Tuned),0.9016,0.8438,0.9643,0.9000,"n_estimators=50, max_depth=5"


In [25]:
# Visual comparison
metrics_data = pd.DataFrame({
    'Model': ['DT Default', 'DT Tuned', 'RF Tuned'] * 4,
    'Metric': ['Accuracy']*3 + ['Precision']*3 + ['Recall']*3 + ['F1 Score']*3,
    'Value': [
        dt_default_metrics['accuracy'], dt_tuned_metrics['accuracy'], rf_metrics['accuracy'],
        dt_default_metrics['precision'], dt_tuned_metrics['precision'], rf_metrics['precision'],
        dt_default_metrics['recall'], dt_tuned_metrics['recall'], rf_metrics['recall'],
        dt_default_metrics['f1'], dt_tuned_metrics['f1'], rf_metrics['f1']
    ]
})

fig = px.bar(metrics_data, x='Metric', y='Value', color='Model', barmode='group',
             title='Model Performance Comparison',
             color_discrete_sequence=['#3498db', '#2ecc71', '#e74c3c'])
fig.update_layout(height=500, width=800, yaxis_range=[0, 1])
fig.show()

### Confusion Matrices

Confusion matrices show the exact counts of true positives, true negatives, false positives, and false negatives for each model.

In [ ]:
# Confusion Matrices for all models
import plotly.figure_factory as ff

def plot_confusion_matrix(y_true, y_pred, title):
    """Create an interactive confusion matrix heatmap."""
    cm = confusion_matrix(y_true, y_pred)
    labels = ['No Disease', 'Disease']
    
    # Create annotated heatmap
    fig = ff.create_annotated_heatmap(
        cm, 
        x=labels, 
        y=labels,
        annotation_text=[[str(y) for y in x] for x in cm],
        colorscale='Blues',
        showscale=True
    )
    
    fig.update_layout(
        title=title,
        xaxis_title='Predicted',
        yaxis_title='Actual',
        width=400,
        height=400
    )
    
    # Fix axis order (actual on y-axis should have "No Disease" at top)
    fig.update_yaxes(autorange='reversed')
    
    return fig, cm

# Plot confusion matrices for each model
fig1, cm1 = plot_confusion_matrix(y_test, y_pred_dt_default, 'Decision Tree (Default) - Confusion Matrix')
fig1.show()
print(f"DT Default - TN: {cm1[0,0]}, FP: {cm1[0,1]}, FN: {cm1[1,0]}, TP: {cm1[1,1]}")

fig2, cm2 = plot_confusion_matrix(y_test, y_pred_dt_tuned, 'Decision Tree (Tuned) - Confusion Matrix')
fig2.show()
print(f"DT Tuned - TN: {cm2[0,0]}, FP: {cm2[0,1]}, FN: {cm2[1,0]}, TP: {cm2[1,1]}")

fig3, cm3 = plot_confusion_matrix(y_test, y_pred_rf, 'Random Forest (Tuned) - Confusion Matrix')
fig3.show()
print(f"RF Tuned - TN: {cm3[0,0]}, FP: {cm3[0,1]}, FN: {cm3[1,0]}, TP: {cm3[1,1]}")

### ROC Curves and AUC Scores

ROC (Receiver Operating Characteristic) curves show the trade-off between true positive rate and false positive rate at various classification thresholds. AUC (Area Under Curve) provides a single metric for overall model discrimination ability.

In [ ]:
# ROC Curves and AUC Scores
from sklearn.metrics import roc_auc_score, roc_curve

# Calculate ROC curves and AUC for each model
models_for_roc = [
    ('Decision Tree (Default)', dt_default, y_pred_dt_default),
    ('Decision Tree (Tuned)', dt_tuned, y_pred_dt_tuned),
    ('Random Forest (Tuned)', rf_tuned, y_pred_rf)
]

fig = go.Figure()
auc_scores = {}

for name, model, y_pred in models_for_roc:
    if hasattr(model, 'predict_proba'):
        y_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_proba)
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        auc_scores[name] = auc
        
        fig.add_trace(go.Scatter(
            x=fpr, y=tpr,
            name=f'{name} (AUC = {auc:.4f})',
            mode='lines'
        ))

# Add diagonal reference line
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    name='Random Classifier',
    mode='lines',
    line=dict(dash='dash', color='gray')
))

fig.update_layout(
    title='ROC Curves Comparison',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    width=700,
    height=500,
    legend=dict(x=0.5, y=0.1)
)

fig.show()

# Print AUC scores
print("\nROC-AUC Scores:")
for name, auc in auc_scores.items():
    print(f"  {name}: {auc:.4f}")

---
## 5. Evaluation & Insights

### Which Model Performed Best?

The **Random Forest (Tuned)** model generally performs best due to its ensemble approach, which reduces variance by averaging predictions from multiple decision trees. Random Forest is less prone to overfitting compared to a single decision tree.

### Most Important Health Indicators

Based on the feature importance analysis, the top predictors of heart disease are:

1. **thal (Thalassemia)** - Blood disorder indicator, strongly associated with cardiovascular health
2. **ca (Major Vessels)** - Number of colored vessels in fluoroscopy, directly indicates arterial blockage
3. **cp (Chest Pain Type)** - Asymptomatic patients often have higher disease rates
4. **oldpeak (ST Depression)** - Exercise-induced ST changes indicate cardiac stress
5. **thalach (Max Heart Rate)** - Lower max heart rate during exercise may indicate cardiac issues

### Hyperparameter Tuning Effects

- **Decision Tree:** Limiting `max_depth` prevents overfitting by creating a simpler, more generalizable model
- **Random Forest:** Tuning `n_estimators` and `max_depth` balances model complexity with generalization

### Insights from Visualizations

1. **Correlation Heatmap:** Shows strong correlations between `cp` (chest pain) and disease, and between `thalach` (max heart rate) and disease
2. **Box Plots:** Patients with heart disease tend to have lower max heart rates and higher ST depression values
3. **Categorical Analysis:** Males have higher disease rates; asymptomatic chest pain is paradoxically associated with higher disease rates

### Ethical Considerations

**False Negatives (Type II Error):** Telling a patient they're healthy when they have disease is dangerous - they may ignore symptoms and delay treatment. This is the more critical error in medical screening.

**False Positives (Type I Error):** Telling a healthy patient they may have disease causes unnecessary anxiety and follow-up tests, but is less dangerous than missing actual disease.

**Recommendation:** In medical screening, prioritize **recall** (sensitivity) to minimize false negatives, even at the cost of some false positives.

### Dataset Limitations

1. **Small sample size:** Only 303 records limits model generalizability
2. **Data age:** The Cleveland dataset was collected in the 1980s; medical practices have changed
3. **Demographics:** May not represent all populations equally
4. **Missing context:** Lifestyle factors (smoking, diet, exercise) not fully captured
5. **Binary simplification:** Original severity levels (1-4) collapsed into binary classification loses granularity

---
## 6. Save Best Model for Deployment

In [26]:
# Determine best model based on F1 score
models = {
    'Decision Tree (Default)': (dt_default, dt_default_metrics['f1']),
    'Decision Tree (Tuned)': (dt_tuned, dt_tuned_metrics['f1']),
    'Random Forest (Tuned)': (rf_tuned, rf_metrics['f1'])
}

best_model_name = max(models, key=lambda x: models[x][1])
best_model = models[best_model_name][0]

print(f"Best performing model: {best_model_name}")
print(f"F1 Score: {models[best_model_name][1]:.4f}")

Best performing model: Random Forest (Tuned)
F1 Score: 0.9000


In [27]:
# Save the best model
joblib.dump(best_model, 'best_model.pkl')
print("Model saved as 'best_model.pkl'")

# Also save the feature names for the web app
joblib.dump(list(X.columns), 'feature_names.pkl')
print("Feature names saved as 'feature_names.pkl'")

Model saved as 'best_model.pkl'
Feature names saved as 'feature_names.pkl'


In [28]:
# Verify model loading
loaded_model = joblib.load('best_model.pkl')
test_prediction = loaded_model.predict(X_test[:1])
print(f"Test prediction successful: {test_prediction[0]}")

Test prediction successful: 0


---
## Summary

This analysis successfully built and compared heart disease prediction models using the Cleveland dataset. The Random Forest classifier achieved the best overall performance, with key predictors including thalassemia status, number of major vessels, and chest pain type. The trained model is saved for deployment in the Heart Risk Advisor web application, which will allow users to input their health metrics and receive an educational risk assessment.

**Important Disclaimer:** This tool is for educational purposes only and should not replace professional medical advice. Always consult healthcare providers for actual medical decisions.